# Tempo ED - Fixed (safe) version

This notebook is a **modified** version of your uploaded `Tempo ED.ipynb`.
I made these safety and compatibility changes:
- Added a top cell that **disables network calls by default**. Set `RUN_NETWORK = True` and provide API keys to enable downloads.
- Replaced timezone-naive `datetime.utcnow()` usages with timezone-aware `datetime.now(timezone.utc)`.
- Wrapped network requests (requests/earthaccess/OpenAQ) in `if RUN_NETWORK:` guards to avoid errors when API keys are missing.
- Added clear placeholders for API keys and instructions.
- Did not execute any network operations. Run cells manually after filling API keys and setting `RUN_NETWORK = True`.

**File saved as:** `/mnt/data/Tempo_ED_fixed.ipynb`


In [ ]:
# Safety controls: set RUN_NETWORK = True only when you have entered API keys and want to perform downloads.
RUN_NETWORK = True  # <-- change to True to enable network calls (and ensure API keys are set)
# Place your API keys below:
# import Contrasenas ignora

API_KEY_OPENAQ = ""      # OpenAQ v3 API key
API_KEY_OWM = ""         # OpenWeatherMap API key
EARTHDATA_USER = ""      # Earthdata login user
EARTHDATA_PASS = ""      # Earthdata login password


In [3]:
if RUN_NETWORK:
    # Network calls are enabled
    # 1. Importar librerías
    import earthaccess
    import os
    import xarray as xr
    import pandas as pd
else:
    print('RUN_NETWORK is False — network calls skipped. Set RUN_NETWORK = True to enable downloads.')


C:\Users\leona\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
if RUN_NETWORK:
    # Network calls are enabled
    # 2. Login interactivo (EDL)
    auth = earthaccess.login(strategy="interactive")
    print("Authenticated?", auth.authenticated)
else:
    print('RUN_NETWORK is False — network calls skipped. Set RUN_NETWORK = True to enable downloads.')


Authenticated? True


In [5]:
# 3. Parámetros de búsqueda
shortname = "TEMPO_NO2_L3"  # ✅ Dataset correcto
date_start = "2025-09-20 00:00:00"
date_end   = "2025-09-28 23:59:59"

print(f"\nSearching granules for {shortname}...")


Searching granules for TEMPO_NO2_L3...


In [6]:
if RUN_NETWORK:
    # Network calls are enabled
    # 4. Buscar granules
    granules = earthaccess.search_data(
        short_name=shortname,
        temporal=(date_start, date_end)
    )

    print(f"Found {len(granules)} granules")

    if not granules:
        raise ValueError("❌ No se encontraron granules en este rango, intenta ampliar fechas")
else:
    print('RUN_NETWORK is False — network calls skipped. Set RUN_NETWORK = True to enable downloads.')


Found 116 granules


In [ ]:
if RUN_NETWORK:
    # Network calls are enabled
    # 5. Descargar un granule al azar
    download_dir = "./tempo_data"
    os.makedirs(download_dir, exist_ok=True)

    # CORRECCIÓN: Pasamos la lista 'granules' directamente sin corchetes extra.
    # earthaccess espera una lista de objetos granule, no una lista que contiene una lista.
    local_paths = earthaccess.download(
        granules=granules, 
        local_path=download_dir
    )

    # Esto imprimirá la ruta del primer archivo descargado.
    print("✅ Granules descargados. La ruta del primer archivo es:", local_paths[0])
else:
    print('RUN_NETWORK is False — network calls skipped. Set RUN_NETWORK = True to enable downloads.')


QUEUEING TASKS | : 100%|██████████| 116/116 [00:00<00:00, 11239.59it/s]
PROCESSING TASKS | :   0%|          | 0/116 [00:00<?, ?it/s]

In [ ]:
# 6. Abrir con xarray
import xarray as xr
ds = xr.open_dataset(local_paths[0])
print(ds)




In [ ]:
#Por si se ocupan revisar las variables acceso rápido:
#print("Variables en este granule:", list(ds.variables))


In [ ]:
 # 7. Explorar variables disponibles
print("Variables en este granule:", list(ds.variables))

# Buscar automáticamente variables que contengan "NO2" en su nombre
no2_vars = [v for v in ds.variables if "NO2" in v.upper()]
print("Variables candidatas de NO2:", no2_vars)

# 8. Si encontramos al menos una, exportamos la primera
if no2_vars:
    var = no2_vars[0]  # agarramos la primera candidata
    print(f"\n✅ Usando variable: {var}")

    df = ds[[var]].to_dataframe().reset_index()
    df.to_csv("tempo_no2_sample.csv", index=False)
    df.to_parquet("tempo_no2_sample.parquet", index=False)
    print("✅ Data exportada a CSV y Parquet")
else:
    print("⚠ No se encontró ninguna variable con 'NO2' en el nombre")